## Connecting to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Importing Library

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import shutil
import numpy as np
import PIL
from PIL import Image
from PIL import ImageOps

import tensorflow as tf
print(tf.__version__)
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Conv2DTranspose
from tensorflow.keras.losses import CategoricalCrossentropy
from keras.layers import UpSampling2D
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split

from tensorflow.python.keras import layers
from keras.models import Model

2.3.0


In [ ]:
# Importing the dataset in the above code using ‘glob’.

NT = glob.glob("/content/drive/My Drive/Images/*T.png") #path of normal tissue
AM = glob.glob("/content/drive/My Drive/Images/*A.png") #path annotated mask

In [ ]:
# Transferring the data to new folder Annotated Tissue
# don't run it again 
# this is done to keep the name of annotated images and normal image same

# base_dir = "/content/drive/My Drive/Images"
# new_dir  = "/content/drive/My Drive/Annotated Tissue/"

# for val in AM:
#   a = val.split('/')[-1].split('.')[0][:-1] + '.png'
#   src_file = os.path.join(base_dir,val.split('/')[-1])
#   shutil.copy(src_file,new_dir)

#   dst_file = os.path.join(new_dir,val.split('/')[-1])
#   new_dst_file_name = os.path.join(new_dir, a)
#   os.rename(dst_file,new_dst_file_name)


In [ ]:
#Transferring the data to new folder Normal Tissue
# don't run it again 
#this is done to keep the name of normal images and normal image same

# new_dir  = "/content/drive/My Drive/Normal Tissue/"

# for val in NT:
#   a = val.split('/')[-1].split('.')[0][:-1] + '.png'
#   src_file = os.path.join(base_dir,val.split('/')[-1])
#   shutil.copy(src_file,new_dir)

#   dst_file = os.path.join(new_dir,val.split('/')[-1])
#   new_dst_file_name = os.path.join(new_dir, a)
#   os.rename(dst_file,new_dst_file_name)

## **Download Data**

In [ ]:
NT = glob.glob("/content/drive/My Drive/Normal Tissue/*.png") #path of normal tissue
AM = glob.glob("/content/drive/My Drive/Annotated Tissue/*.png") #path annotated mask

#function for getting the data
def get_data(NT,AM):

  '''
  Here we have initialised two lists, converting the tissue images and the annotated (labels) images
  appending them to ‘normal_image’ and ‘annotated_image’ respectively and returning the list.
  
  The imported data is also being converted to numpy array 
  As well as also showing image of some of the imported data
  '''

  normal_image = []
  annotated_image = []
  normal_image_np = []
  annotated_image_np = []

  for image, mask in zip(sorted(NT), sorted(AM)):
    normal_image.append(image)
    annotated_image.append(mask)
    normal_image_np.append(np.array(Image.open(image)))
    annotated_image_np.append(np.array(Image.open(mask).convert('RGBA'))) 

  #sorting the data 
  annotated_image.sort()
  normal_image.sort()

  normal_image_ns = np.asarray(normal_image_np)
  annotated_image_ns = np.asarray(annotated_image_np)

  x = np.asarray(normal_image_ns, dtype=np.float32)/255 #  Normalization of data
  y = np.asarray(annotated_image_ns, dtype=np.float32)/255  #  Normalization of data 

  # x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 4)
  # y = y.reshape(y.shape[0], y.shape[1], y.shape[2], 4) # mode=P that means it is palettised
  
  #plotting the dataset
  figsize=(25,5)
  fig, ax = plt.subplots(nrows=2, ncols=5, figsize=figsize)

  #Tissue Image
  for count,ele in enumerate(normal_image[0:5]): 
    
    #plotting the tiusse image
    img = mpimg.imread(ele)
    ax[0,count].set(title = ele.split('/')[-1])
    ax[0,count].imshow(img)
  
  #Annotated Image
  for count,ele in enumerate(annotated_image[0:5]): 
    #plotting the tiusse image
    img = mpimg.imread(ele)
    ax[1,count].set(title = ele.split('/')[-1])
    ax[1,count].imshow(img)
  
  fig.tight_layout() 
  
  return normal_image,annotated_image,x,y

In [ ]:
#get data 
normal_image,annotated_image,x,y = get_data(NT,AM)

In [ ]:
#data split
def data_split(x,y,test_size):

  x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=test_size, random_state=1)

  print("x_train: ", x_train.shape)
  print("y_train: ", y_train.shape)
  print("x_val: ", x_val.shape)
  print("y_val: ", y_val.shape)

  return x_train, x_val, y_train, y_val

In [ ]:
#getting the splitted data

x_train, x_val, y_train, y_val = data_split(x,y,test_size=0.2)

x_train:  (224, 512, 512, 4)
y_train:  (224, 512, 512, 4)
x_val:  (56, 512, 512, 4)
y_val:  (56, 512, 512, 4)


## Function for creating U-Net architecture

In [ ]:
#convolutional block
def conv_block(input_tensor, num_filters):
  encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor) #conv2d layer is used here to provide i/p tensor where kernel size 3 X 3
  encoder = layers.BatchNormalization()(encoder)
  #encoder = layers.Dropout(0.3)(encoder) 
  encoder = layers.Activation('relu')(encoder) #activation is used here to map output of previous layer to i/p of next layer, Relu passes same output 
                                               #incase of positive i/p and passes zero in case negative i/p 

  encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
  encoder = layers.BatchNormalization()(encoder)
  #encoder = layers.Dropout(0.3)(encoder) 
  encoder = layers.Activation('relu')(encoder)

  return encoder

#encoder
def encoder_block(input_tensor, num_filters):
  encoder = conv_block(input_tensor, num_filters)
  encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder) #maxpooling is used to reduce the workload of upcoming layers(downsampling)
  encoder_pool = layers.Dropout(0.2)(encoder_pool)  #20 percent of the nodes were dropped from the next layer
  return encoder_pool, encoder

#decoder
def decoder_block(input_tensor, concat_tensor, num_filters):
  decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)# for doubling input image size(upsampling)
  decoder = layers.concatenate([concat_tensor, decoder], axis=-1) 
  #decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  decoder = layers.Dropout(0.2)(decoder)

  decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  #decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  decoder = layers.Dropout(0.2)(decoder)

  decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  #decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  decoder = layers.Dropout(0.2)(decoder)
  
  return decoder

## Calling U-Net functions to create architecture

In [ ]:
def get_unet(img_shape):

#img_shape = x_train[0].shape

# Downsampling path
  inputs = layers.Input(shape=img_shape)
  encoder0_pool, encoder0 = encoder_block(inputs,32) #32 these are feature space
  encoder1_pool, encoder1 = encoder_block(encoder0_pool,64)
  encoder2_pool, encoder2 = encoder_block(encoder1_pool,128)
  encoder3_pool, encoder3 = encoder_block(encoder2_pool,256)
  encoder4_pool, encoder4 = encoder_block(encoder3_pool,512)

  #center
  center = conv_block(encoder4_pool,1024)
  
# Upampling path 
  decoder4 = decoder_block(center,encoder4,512)
  decoder3 = decoder_block(decoder4,encoder3,256)
  decoder2 = decoder_block(decoder3,encoder2,128)
  decoder1 = decoder_block(decoder2,encoder1,64)
  decoder0 = decoder_block(decoder1,encoder0,32)

  #output layer
  outputs = layers.Conv2D(4,(1,1),activation='sigmoid')(decoder0)

  #from keras.optimizers import Adam, SGD
  
  model = Model(inputs=[inputs], outputs=[outputs])
  model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])
  model.summary()

  return model

# model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.99), metrics=['accuracy'])
# model.summary()


#optimizer=SGD(lr=0.01, momentum=0.99),
#tf.keras.optimizers.Adam(learning_rate=1e-4)
#"adam"

In [ ]:
#preparing the unet model

model = get_unet(img_shape=x_train[0].shape)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 4) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 32) 1184        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 512, 512, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

## **Model Training**

In [ ]:
def train(model,x_train, y_train):

  es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode = 'min', patience=8)
  #mc = tf.keras.callbacks.ModelCheckpoint('NTAMmodel.h5', monitor='val_accuracy',mode = 'max', verbose=1, save_best_only=True)

  history = model.fit(x=x_train, y=y_train,batch_size=8,callbacks = es,epochs=50, verbose=1, validation_split=0.2)

  return history

In [ ]:
#training the models
history = train(model=model,x_train=x_train,y_train=y_train)

Epoch 1/50
23/23 [==============================] - 53s 2s/step - loss: 2.8443 - accuracy: 0.1193 - val_loss: 2.8609 - val_accuracy: 0.0260
Epoch 2/50
23/23 [==============================] - 40s 2s/step - loss: 2.1766 - accuracy: 0.3655 - val_loss: 2.6743 - val_accuracy: 0.1211
Epoch 3/50
23/23 [==============================] - 40s 2s/step - loss: 2.0073 - accuracy: 0.3737 - val_loss: 2.4017 - val_accuracy: 0.0901
Epoch 4/50
23/23 [==============================] - 40s 2s/step - loss: 1.9412 - accuracy: 0.3509 - val_loss: 2.3214 - val_accuracy: 0.0268
Epoch 5/50
23/23 [==============================] - 40s 2s/step - loss: 1.8978 - accuracy: 0.3207 - val_loss: 2.2243 - val_accuracy: 0.0260
Epoch 6/50
23/23 [==============================] - 40s 2s/step - loss: 1.9493 - accuracy: 0.2822 - val_loss: 2.3796 - val_accuracy: 0.0257
Epoch 7/50
23/23 [==============================] - 40s 2s/step - loss: 1.8974 - accuracy: 0.2725 - val_loss: 2.1294 - val_accuracy: 0.0256
Epoch 8/50
23/23 [==

## **Model Evaluation**

In [ ]:
#evaluating the model
score = model.evaluate(x=x_val, y=y_val, verbose=1)

2/2 [==============================] - 1s 647ms/step - loss: 1.8302 - accuracy: 0.0715


## **Ploting History**

In [ ]:
def plot_history(history,score):
  
  print("Test Score:", score[0])
  print("Test Accuracy:", score[1])

  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])

  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train','test'], loc='upper left')
  plt.show()
  plt.savefig('Accuracy.png')

  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])

  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train','test'], loc='upper left')
  plt.show()
  plt.savefig('Loss.png')

In [ ]:
 #plotting the hostory
plot_history(history,score)

## **Model Prediction**

In [ ]:
#predition
y_pred = model.predict(x_val)

In [ ]:
#dice coefficient
def dice_coeff(y_true, y_pred, smooth = 1):

  y_true_f = tf.reshape(y_val,[-1])
  y_pred_f = tf.reshape(y_pred,[-1])
  intersection = tf.reduce_sum(y_true_f * y_pred_f )
  score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

  return score

In [ ]:
score = dice_coeff(y_val, y_pred, smooth = 1)
print(score)

tf.Tensor(0.90060884, shape=(), dtype=float32)


In [ ]:
plt.imshow(x_val[2])

In [ ]:
plt.imshow(y_val[2])

In [ ]:
plt.imshow(y_pred[2])

## **Plotting the Images**

In [ ]:
def plot_imgs(org_imgs, mask_imgs,pred_imgs=None,nm_img_to_plot=15,figsize=4,alpha=0.5):
    '''
    Image plotting for semantic segmentation data.
    Last column is always an overlay of ground truth or prediction
    depending on what was provided as arguments.
    '''
    if nm_img_to_plot > org_imgs.shape[0]:
        nm_img_to_plot = org_imgs.shape[0]
    im_id = 0
    org_imgs_size = org_imgs.shape[1]

    org_imgs = reshape_arr(org_imgs)
    mask_imgs = reshape_arr(mask_imgs)
    if  not (pred_imgs is None):
        cols = 4
        pred_imgs = reshape_arr(pred_imgs)
    else:
        cols = 3

        
    fig, axes = plt.subplots(nm_img_to_plot, cols, figsize=(cols*figsize, nm_img_to_plot*figsize))
    axes[0, 0].set_title("original", fontsize=15) 
    axes[0, 1].set_title("ground truth", fontsize=15)
    if not (pred_imgs is None):
        axes[0, 2].set_title("prediction", fontsize=15) 
        axes[0, 3].set_title("overlay", fontsize=15) 
    else:
        axes[0, 2].set_title("overlay", fontsize=15) 
    for m in range(0, nm_img_to_plot):
        axes[m, 0].imshow(org_imgs[im_id]) #, cmap=get_cmap(org_imgs)
        axes[m, 0].set_axis_off()
        axes[m, 1].imshow(mask_imgs[im_id]) #, cmap=get_cmap(mask_imgs)
        axes[m, 1].set_axis_off()        
        if not (pred_imgs is None):
            axes[m, 2].imshow(pred_imgs[im_id]) #, cmap=get_cmap(pred_imgs)
            axes[m, 2].set_axis_off()
            axes[m, 3].imshow(org_imgs[im_id]) #,cmap=get_cmap(org_imgs)
            axes[m, 3].imshow(pred_imgs[im_id])
            axes[m, 3].set_axis_off()
        else:
            axes[m, 2].imshow(org_imgs[im_id], cmap=get_cmap(org_imgs))
            axes[m, 2].imshow(mask_imgs[im_id])
            axes[m, 2].set_axis_off()
        im_id += 1

    plt.show()
  
def reshape_arr(arr):
    if arr.ndim == 3:
        return arr
    elif arr.ndim == 4:
        if arr.shape[3] == 4:
            return arr
        elif arr.shape[3] == 1:
            return arr.reshape(arr.shape[0], arr.shape[1], arr.shape[2],4)

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15) #d=0.2,e=50 

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15) #d=0.2,e=50 

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15) #d=0.2,e=50 

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15) #d=0.2,e=50 

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15) #d=0.2,e=50 

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15) #d=0.2,e=50 

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15) #d=0.2,e=50 with BN(batch normalisation in decoder)

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15) #d=0.2,e=50

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15) #d=0.2,e=30

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15)

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15)

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15)

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15)

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15)

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15)

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15)

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15)

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15) #when dropout was present in conv2D

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15)

In [ ]:
plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=15)